In [10]:
from bs4 import BeautifulSoup
import os
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from os import walk
#from django.utils.html import strip_tags
import string
import re
import pandas as pd
from urllib.request import Request, urlopen
import lxml
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

This piece of code will provide you lyrics from www.lyrics.com for a given artist:

In [ ]:
#enter the artist's name like this: artist="Queen" or artist = "The+Beatles"

artist = "" 

os.mkdir(f"/work/{artist}",mode = 0o666) 

url_0 = "https://www.lyrics.com/artist/" + artist
html_0 = requests.get(url_0)

soup_0 = BeautifulSoup(html_0.text)

raw_url = soup_0.find_all("a", attrs={"href":(lambda L: L and L.startswith("/lyric/"))})

for i in range(100):         # there are doubletons so we collect more than 50 texts
  str_i = str(raw_url[i])
  i1 = str_i.find('"')+1
  i2 = str_i.find('"',i1)
  song_url = "https://www.lyrics.com" + str_i[i1:i2]
  #print(song_url)
  i1 = song_url.find(artist)
  i2 = song_url.find('/', i1)+1
  title =song_url[i2:]
  f_title = title + ".txt"

  html_1 = requests.get(song_url)
  soup_1 = BeautifulSoup(html_1.text)
  lyric_1 = soup_1.find_all("pre", attrs={"class":"lyric-body"})
  os.chdir(f"/work/{artist}")
  try:
    with open(f_title, "w") as f:
      f.write(lyric_1[0].get_text())
  except IndexError:
    pass

Now&nbsp;the&nbsp;lyrics&nbsp;are&nbsp;stored&nbsp;in&nbsp;folders&nbsp;named&nbsp;after&nbsp;artists. Read&nbsp;them&nbsp;into&nbsp;a&nbsp;dictionary&nbsp;then&nbsp;make&nbsp;a&nbsp;data&nbsp;frame

In [11]:
artist_list = ["Madonna", "Queen", "Sting", "The+Beatles"]

df_lyrics = pd.DataFrame(columns=['artist'])

for artist in artist_list:
    lyrics_dict= {}

    for file_name in os.listdir(f"/work/{artist}/"):
        lyric = open(f"/work/{artist}/"+file_name).read()
        lyrics_dict[lyric] = artist
    
    for i in lyrics_dict:
        df_lyrics.at[i,'artist'] = lyrics_dict.get(i)

df_lyrics.rename_axis('lyric', inplace=True)
df_lyrics.reset_index(inplace=True)

df_lyrics.head()

,lyric,artist
0,Hey Mister DJ\nPut a record on\nI wanna dance ...,Madonna
1,Woke up this morning feeling that you were gon...,Madonna
2,Sooner or later you're gonna be mine\nSooner o...,Madonna
3,"Who's that girl, who's that girl\n\nWhen you s...",Madonna
4,"Drink me up Champagne Rosé (oh, Huncho did it?...",Madonna


In [12]:
def clean_text(text):
  # get rid of all unwanted punctuation and digits
  text = text.replace("´", "'")

  digi_punct = "[^a-zA-Z' ]"
  text = re.sub(digi_punct, " ", text)
  text = " ".join(text.split())
  text = text.lower()
  return text

df_lyrics['lyric'] = df_lyrics['lyric'].apply(lambda x:clean_text(x))
df_lyrics.head()

,lyric,artist
0,hey mister dj put a record on i wanna dance wi...,Madonna
1,woke up this morning feeling that you were gon...,Madonna
2,sooner or later you're gonna be mine sooner or...,Madonna
3,who's that girl who's that girl when you see h...,Madonna
4,drink me up champagne ros oh huncho did it it'...,Madonna


In [13]:
import nltk
#nltk.download('popular')

def TokenizeWords(text):
    return (nltk.word_tokenize(text))

df_lyrics['lyric'] = df_lyrics['lyric'].apply(lambda x:TokenizeWords(x))

df_lyrics.head()


,lyric,artist
0,"[hey, mister, dj, put, a, record, on, i, wan, ...",Madonna
1,"[woke, up, this, morning, feeling, that, you, ...",Madonna
2,"[sooner, or, later, you, 're, gon, na, be, min...",Madonna
3,"[who, 's, that, girl, who, 's, that, girl, whe...",Madonna
4,"[drink, me, up, champagne, ros, oh, huncho, di...",Madonna


In [15]:
!pip install spacy
import spacy
# Import of English rules
nlp = spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [7]:
# additional data cleaning: remove stopwords, lemmatize afterwards

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')

stpwrds_list = list(stopwords.words("english"))
ext_list = list("?!.,;'´()") + \
['"', "''", "``", "'s", "I", "'ll", "'m", "'re","'ve", "'nt", "n't","'d"]

lemmatizer = WordNetLemmatizer()

for i in range(len(df_lyrics)):
    word_list_0 = df_lyrics['lyric'][i]
    word_list_1 = []
    for w in word_list_0:
        w_lower = w.lower()
        if w_lower not in (stpwrds_list + ext_list):
            #w_ = lemmatizer.lemmatize(w_lower, pos="a")
            #w_ = lemmatizer.lemmatize(w_, pos="v")
            #w_ = lemmatizer.lemmatize(w_, pos="n")
            w_ = nlp(w_lower).lemma_
            
            word_list_1.append(w_)
   
    df_lyrics['lyric'][i] = word_list_1

df_lyrics.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2021-10-01 15:35:06.472882: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-01 15:35:06.472938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [1]:
df_lyrics

NameError: name 'df_lyrics' is not defined

In [ ]:
#transform lists of strings (single words) into one joined string having a space between the words

df_lyrics['lyric'] = df_lyrics['lyric'].apply(lambda x: ' '.join(x))
df_lyrics.head()

,lyric,artist
0,hey mister dj put record wan na dance baby hey...,Madonna
1,wake morning feel go think awhile finally move...,Madonna
2,sooner late gon na mine sooner late gon na fin...,Madonna
3,girl girl see say prayer kiss heart goodbye tr...,Madonna
4,drink champagne ro oh huncho game murda please...,Madonna


In [ ]:
X_remain, X_test, y_remain, y_test = train_test_split(df_lyrics["lyric"],
                                                    df_lyrics["artist"],
                                                    test_size=0.2,
                                                    random_state=42)


X_train, X_val, y_train, y_val = train_test_split(X_remain, y_remain,
                                                    test_size=0.2,
                                                    random_state=42)


top = [X_remain, X_test, y_remain, y_test, X_train, X_val, y_train, y_val]
for x in top:
    print(x.shape)

(170,)
(43,)
(170,)
(43,)
(136,)
(34,)
(136,)
(34,)


In [ ]:
X_train.head()

4      drink champagne ro oh huncho game murda please...
58     new purple shoe amazin people next door rock '...
109    another obituary paper today one list already ...
156    besame besame mucho time bring kiss hear music...
209    wan na lover baby wan na man wan na lover baby...
Name: lyric, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vectorizer = CountVectorizer()

bag_of_words = count_vectorizer.fit_transform(X_train)

print(type(bag_of_words[0]))
print(bag_of_words.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(136, 2454)


In [ ]:
print(bag_of_words[62])

  (0, 1463)	1
  (0, 1582)	4
  (0, 2436)	1
  (0, 2377)	5
  (0, 1963)	1
  (0, 1192)	1
  (0, 1421)	1
  (0, 1145)	4
  (0, 1235)	24
  (0, 54)	4
  (0, 2242)	4
  (0, 1966)	3


In [ ]:
pipe = make_pipeline(TfidfVectorizer(), LogisticRegression())
cross_val_score(pipe, X_remain, y_remain).mean()

0.5411764705882354

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fb01898c-b7e0-4a0a-bfc6-7e769492b0ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>